## Examples for TimeSeriesValue and TimeSeries Classes

### Setup

In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
import math
from datetime import timedelta

import pandas as pd

from hec import (
    Combine,
    CwmsDataStore,
    HecTime,
    Quality,
    Select,
    TimeSeriesException,
    TimeSeriesValue,
    TimeSpan,
)
from hec import UnitQuantity as UQ

### TimeSeriesValue Examples

In [2]:
tsv = TimeSeriesValue("14Oct2024 10:55", UQ(12.3, "ft"), Quality("okay"))
print(f"repr    = {repr(tsv)}")
print(f"str     = {tsv}")
print(f"time    = {tsv.time}")
print(f"value   = {tsv.value}")
print(f"quality = {tsv.quality.text}")
print("tsv.time += timedelta(minutes = 65)")
tsv.time += timedelta(minutes=65)
print("tsv.value += .7")
tsv.value += 0.7
print('tsv.quality = Qual("missing").set_protection(1)')
tsv.quality = Quality("missing").set_protection(1)
print(f"repr    = {repr(tsv)}")
print(f"str     = {tsv}")
print(f"time    = {tsv.time}")
print(f"value   = {tsv.value}")
print(f"quality = {tsv.quality.text}")

repr    = TimeSeriesValue(HecTime([2024, 10, 14, 10, 55, 0], MINUTE_GRANULARITY), UnitQuantity(12.3, 'ft'), Quality(3))
str     = (2024-10-14T10:55:00, 12.3 ft, o)
time    = 2024-10-14T10:55:00
value   = 12.3 ft
quality = Screened Okay No_Range Original None None None Unprotected
tsv.time += timedelta(minutes = 65)
tsv.value += .7
tsv.quality = Qual("missing").set_protection(1)
repr    = TimeSeriesValue(HecTime([2024, 10, 14, 12, 0, 0], MINUTE_GRANULARITY), UnitQuantity(13.0, 'ft'), Quality(-2147483643))
str     = (2024-10-14T12:00:00, 13.0 ft, M)
time    = 2024-10-14T12:00:00
value   = 13.0 ft
quality = Screened Missing No_Range Original None None None Protected


### Retrieve TimeSeries and Print Info

In [3]:
db = CwmsDataStore.open(office="SWT", start_time=HecTime.now() - 1440)
elev = db.retrieve("Keys.Elev.Inst.1Hour.0.Ccp-rev")
print(f"repr            = {repr(elev)}")
print(f"str             = {elev}")
print(f"has_selection   = {elev.has_selection}")
print(f"selection_state = {elev.selection_state}")
times = list(map(str, elev.times))
print(f"times     = {times}")
print(f"values    = {elev.values}")
print(f"qualities = {elev.qualities}")
print(f"data = \n{elev.data}")
assert elev.data is not None

repr            = <TimeSeries('KEYS.Elev.Inst.1Hour.0.Ccp-Rev'): 24 values, unit=ft>
str             = KEYS.Elev.Inst.1Hour.0.Ccp-Rev 24 values in ft
has_selection   = False
selection_state = SelectionState.TRANSIENT
times     = ['2025-10-21 12:00:00-05:00', '2025-10-21 13:00:00-05:00', '2025-10-21 14:00:00-05:00', '2025-10-21 15:00:00-05:00', '2025-10-21 16:00:00-05:00', '2025-10-21 17:00:00-05:00', '2025-10-21 18:00:00-05:00', '2025-10-21 19:00:00-05:00', '2025-10-21 20:00:00-05:00', '2025-10-21 21:00:00-05:00', '2025-10-21 22:00:00-05:00', '2025-10-21 23:00:00-05:00', '2025-10-22 00:00:00-05:00', '2025-10-22 01:00:00-05:00', '2025-10-22 02:00:00-05:00', '2025-10-22 03:00:00-05:00', '2025-10-22 04:00:00-05:00', '2025-10-22 05:00:00-05:00', '2025-10-22 06:00:00-05:00', '2025-10-22 07:00:00-05:00', '2025-10-22 08:00:00-05:00', '2025-10-22 09:00:00-05:00', '2025-10-22 10:00:00-05:00', '2025-10-22 11:00:00-05:00']
values    = [725.1099999999999, 725.0799999999999, 725.0899999999999, 725.

### Index TimeSeries Object

In [4]:
print("\nIndex by int: elev[1]")
print(elev[1].data)
print(f"\nIndex by str: elev['{times[2]}']")
print(elev[times[2]].data)
print(f"\nIndex by HecTime: elev[{repr(HecTime(times[3]))}]")
print(elev[HecTime(times[3])].data)
print(f"\nIndex by datetime: elev[{repr(HecTime(times[4]).datetime())}]")
print(elev[HecTime(times[4]).datetime()].data)


Index by int: elev[1]
value      725.08
quality      0.00
Name: 2025-10-21 13:00:00-05:00, dtype: float64

Index by str: elev['2025-10-21 14:00:00-05:00']
value      725.09
quality      0.00
Name: 2025-10-21 14:00:00-05:00, dtype: float64

Index by HecTime: elev[HecTime([2025, 10, 21, 15, 0, 0], MINUTE_GRANULARITY).label_as_time_zone("America/Chicago")]
value      725.05
quality      0.00
Name: 2025-10-21 15:00:00-05:00, dtype: float64

Index by datetime: elev[datetime.datetime(2025, 10, 21, 16, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Chicago'))]
value      725.01
quality      0.00
Name: 2025-10-21 16:00:00-05:00, dtype: float64


### Slicing TimeSeries Objects

In [5]:
start = -10
stop = -4
print(f"Slice = [{start}:{stop}:1]")
print(f"Python slicing: excludes {stop}")
elev.slice_stop_exclusive = True
print(elev[start:stop].data)
print(f"\nDataFrame slicing: includes {stop}")
elev.slice_stop_exclusive = False
print(elev[start:stop].data)
print(f"\nSlicing with step: get every other row")
print(f"Slice = [{start}:{stop}:2]")
print(elev[start:stop:2].data)

Slice = [-10:-4:1]
Python slicing: excludes -4
                            value  quality
time                                      
2025-10-21 12:00:00-05:00  725.11        0
2025-10-21 13:00:00-05:00  725.08        0
2025-10-21 14:00:00-05:00  725.09        0
2025-10-21 15:00:00-05:00  725.05        0
2025-10-21 16:00:00-05:00  725.01        0
2025-10-21 17:00:00-05:00  724.96        0
2025-10-21 18:00:00-05:00  724.94        0
2025-10-21 19:00:00-05:00  724.89        0
2025-10-21 20:00:00-05:00  724.86        0
2025-10-21 21:00:00-05:00  724.86        0
2025-10-21 22:00:00-05:00  724.87        0
2025-10-21 23:00:00-05:00  724.87        0
2025-10-22 00:00:00-05:00  724.90        0
2025-10-22 01:00:00-05:00  724.90        0
2025-10-22 02:00:00-05:00  724.92        0
2025-10-22 03:00:00-05:00  724.92        0
2025-10-22 04:00:00-05:00  724.94        0
2025-10-22 05:00:00-05:00  724.95        0
2025-10-22 06:00:00-05:00  724.96        0
2025-10-22 07:00:00-05:00  724.97        0

DataFr

### Modification of Values and Qualitties

In [6]:
print("Modify directly in DataFrame")
elev.data.loc[elev.index_of(start), "value"] = 1000
elev.data.loc[elev.index_of(stop), "quality"] = Quality("Missing").code
print(elev[start:stop].data)
print("\nModify using API")
elev.iselect(lambda tsv: tsv.quality.score < 1)
elev.iset_value(math.nan)
print(elev[start:stop].data)

Modify directly in DataFrame
                             value  quality
time                                       
2025-10-21 12:00:00-05:00   725.11        0
2025-10-21 13:00:00-05:00   725.08        0
2025-10-21 14:00:00-05:00   725.09        0
2025-10-21 15:00:00-05:00   725.05        0
2025-10-21 16:00:00-05:00   725.01        0
2025-10-21 17:00:00-05:00   724.96        0
2025-10-21 18:00:00-05:00   724.94        0
2025-10-21 19:00:00-05:00   724.89        0
2025-10-21 20:00:00-05:00   724.86        0
2025-10-21 21:00:00-05:00   724.86        0
2025-10-21 22:00:00-05:00   724.87        0
2025-10-21 23:00:00-05:00   724.87        0
2025-10-22 00:00:00-05:00   724.90        0
2025-10-22 01:00:00-05:00   724.90        0
2025-10-22 02:00:00-05:00  1000.00        0
2025-10-22 03:00:00-05:00   724.92        0
2025-10-22 04:00:00-05:00   724.94        0
2025-10-22 05:00:00-05:00   724.95        0
2025-10-22 06:00:00-05:00   724.96        0
2025-10-22 07:00:00-05:00   724.97        0
202

### Working With Time Zones

In [7]:
new_ts = elev[start:stop]
print("\nRemove time zone")
new_ts.label_as_time_zone(None)
print(new_ts.data)
print(f"time zone = {new_ts.time_zone}")
print("\nSet time zone to US/Central")
new_ts.label_as_time_zone("US/Central")
print(new_ts.data)
print(f"time zone = {new_ts.time_zone}")
print("\nSet time (don't convert) zone to back to UTC")
print(
    "...first with on_already_set=2 (error if already set and setting to another tz)\n"
)
try:
    new_ts.label_as_time_zone("UTC", on_already_set=2)
except TimeSeriesException as e:
    print(f"TimeSeriesException: {' '.join(e.args)}")
print("\n...next with on_already_set=0 (no errors or warnings, just do what I say)")
new_ts.label_as_time_zone("UTC", on_already_set=0)
print(new_ts.data)
print(f"time zone = {new_ts.time_zone}")
print("\n...can also use on_already_set=1 to allow, but generate a warning")
print("\nNow convert (don't set) to US/Central")
new_ts.convert_to_time_zone("US/Central")
print(new_ts.data)
print(f"time zone = {new_ts.time_zone}")


Remove time zone
                       value  quality
time                                 
2025-10-21 12:00:00   725.11        0
2025-10-21 13:00:00   725.08        0
2025-10-21 14:00:00   725.09        0
2025-10-21 15:00:00   725.05        0
2025-10-21 16:00:00   725.01        0
2025-10-21 17:00:00   724.96        0
2025-10-21 18:00:00   724.94        0
2025-10-21 19:00:00   724.89        0
2025-10-21 20:00:00   724.86        0
2025-10-21 21:00:00   724.86        0
2025-10-21 22:00:00   724.87        0
2025-10-21 23:00:00   724.87        0
2025-10-22 00:00:00   724.90        0
2025-10-22 01:00:00   724.90        0
2025-10-22 02:00:00  1000.00        0
2025-10-22 03:00:00   724.92        0
2025-10-22 04:00:00   724.94        0
2025-10-22 05:00:00   724.95        0
2025-10-22 06:00:00   724.96        0
2025-10-22 07:00:00   724.97        0
2025-10-22 08:00:00      NaN        5
time zone = None

Set time zone to US/Central
                             value  quality
time              

### Time Shifting

In [8]:
print("Original data")
print(new_ts.data)
print("\nShifted back 3 intervals using integer")
new_ts <<= 3
print(new_ts.data)
print("\nShifted forward 3 months using TimeSpan")
new_ts >>= TimeSpan("P3M")
print(new_ts.data)
print("\nShifted back 30 minutes using timedelta")
new_ts <<= timedelta(minutes=30)
print(new_ts.data)

Original data
                             value  quality
time                                       
2025-10-21 12:00:00+00:00   725.11        0
2025-10-21 13:00:00+00:00   725.08        0
2025-10-21 14:00:00+00:00   725.09        0
2025-10-21 15:00:00+00:00   725.05        0
2025-10-21 16:00:00+00:00   725.01        0
2025-10-21 17:00:00+00:00   724.96        0
2025-10-21 18:00:00+00:00   724.94        0
2025-10-21 19:00:00+00:00   724.89        0
2025-10-21 20:00:00+00:00   724.86        0
2025-10-21 21:00:00+00:00   724.86        0
2025-10-21 22:00:00+00:00   724.87        0
2025-10-21 23:00:00+00:00   724.87        0
2025-10-22 00:00:00+00:00   724.90        0
2025-10-22 01:00:00+00:00   724.90        0
2025-10-22 02:00:00+00:00  1000.00        0
2025-10-22 03:00:00+00:00   724.92        0
2025-10-22 04:00:00+00:00   724.94        0
2025-10-22 05:00:00+00:00   724.95        0
2025-10-22 06:00:00+00:00   724.96        0
2025-10-22 07:00:00+00:00   724.97        0
2025-10-22 08:00:0

### Scalar Math

In [9]:
print("Add 1000")
new_ts += 1000
print(new_ts.data)
print("\nDivide by 2")
new_ts /= 2
print(new_ts.data)
print("\nApply lambda v: v - v % 0.01")
new_ts.imap(lambda v: v - v % 0.01)
print(new_ts.data)

Add 1000
                             value  quality
time                                       
2026-01-21 08:30:00+00:00  1725.11        0
2026-01-21 09:30:00+00:00  1725.08        0
2026-01-21 10:30:00+00:00  1725.09        0
2026-01-21 11:30:00+00:00  1725.05        0
2026-01-21 12:30:00+00:00  1725.01        0
2026-01-21 13:30:00+00:00  1724.96        0
2026-01-21 14:30:00+00:00  1724.94        0
2026-01-21 15:30:00+00:00  1724.89        0
2026-01-21 16:30:00+00:00  1724.86        0
2026-01-21 17:30:00+00:00  1724.86        0
2026-01-21 18:30:00+00:00  1724.87        0
2026-01-21 19:30:00+00:00  1724.87        0
2026-01-21 20:30:00+00:00  1724.90        0
2026-01-21 21:30:00+00:00  1724.90        0
2026-01-21 22:30:00+00:00  2000.00        0
2026-01-21 23:30:00+00:00  1724.92        0
2026-01-22 00:30:00+00:00  1724.94        0
2026-01-22 01:30:00+00:00  1724.95        0
2026-01-22 02:30:00+00:00  1724.96        0
2026-01-22 03:30:00+00:00  1724.97        0
2026-01-22 04:30:00+00:

### TimeSeries Math

In [10]:
new_ts1 = elev[:10]
new_ts2 = new_ts1 >> 2
print(new_ts1.data)
print(new_ts2.data)
print((new_ts1 + new_ts2).data)

                            value  quality
time                                      
2025-10-21 12:00:00-05:00  725.11        0
2025-10-21 13:00:00-05:00  725.08        0
2025-10-21 14:00:00-05:00  725.09        0
2025-10-21 15:00:00-05:00  725.05        0
2025-10-21 16:00:00-05:00  725.01        0
2025-10-21 17:00:00-05:00  724.96        0
2025-10-21 18:00:00-05:00  724.94        0
2025-10-21 19:00:00-05:00  724.89        0
2025-10-21 20:00:00-05:00  724.86        0
2025-10-21 21:00:00-05:00  724.86        0
2025-10-21 22:00:00-05:00  724.87        0
                            value  quality
time                                      
2025-10-21 14:00:00-05:00  725.11        0
2025-10-21 15:00:00-05:00  725.08        0
2025-10-21 16:00:00-05:00  725.09        0
2025-10-21 17:00:00-05:00  725.05        0
2025-10-21 18:00:00-05:00  725.01        0
2025-10-21 19:00:00-05:00  724.96        0
2025-10-21 20:00:00-05:00  724.94        0
2025-10-21 21:00:00-05:00  724.89        0
2025-10-21 

### Selecting Rows

In [11]:
new_ts1.iselect(Select.ALL)
print(new_ts1.data)
new_ts1.iselect(Select.NONE)
print(new_ts1.data)
new_ts1.iselect(lambda tsv: tsv.value > 719.5)
print(new_ts1.data)
new_ts1.iselect(Select.INVERT)
print(new_ts1.data)
print(new_ts1.select(lambda tsv: tsv.value < 719.7).data, Combine.XOR)
print(new_ts1.select(lambda tsv: tsv.time > elev[5].tsv[0].time).data)

                            value  quality
time                                      
2025-10-21 12:00:00-05:00  725.11        0
2025-10-21 13:00:00-05:00  725.08        0
2025-10-21 14:00:00-05:00  725.09        0
2025-10-21 15:00:00-05:00  725.05        0
2025-10-21 16:00:00-05:00  725.01        0
2025-10-21 17:00:00-05:00  724.96        0
2025-10-21 18:00:00-05:00  724.94        0
2025-10-21 19:00:00-05:00  724.89        0
2025-10-21 20:00:00-05:00  724.86        0
2025-10-21 21:00:00-05:00  724.86        0
2025-10-21 22:00:00-05:00  724.87        0
                            value  quality
time                                      
2025-10-21 12:00:00-05:00  725.11        0
2025-10-21 13:00:00-05:00  725.08        0
2025-10-21 14:00:00-05:00  725.09        0
2025-10-21 15:00:00-05:00  725.05        0
2025-10-21 16:00:00-05:00  725.01        0
2025-10-21 17:00:00-05:00  724.96        0
2025-10-21 18:00:00-05:00  724.94        0
2025-10-21 19:00:00-05:00  724.89        0
2025-10-21 

u:\Devl\git\hec-python-library\hec\timeseries.py:7554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "selected"] = data.apply(
